In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import plotly.express as px
import plotly.graph_objects as go
#pd.options.display.float_format = '{:.4f}'.format

In [21]:
df_results = pd.read_csv('Electoral_College.csv')
df_results = df_results.drop(df_results.columns[4:], axis = 1)
df_results_1976 = df_results[df_results['Year'] >= 1976]
df_results_1976.reset_index(inplace=True, drop=True)
df_results_1976.insert(2,'Population',np.zeros(612))
df_results_1976.insert(4,'VotesT',np.zeros(612))
df_results_1976.insert(len(df_results_1976.columns), 'WinnerVotes', np.zeros(612))
df_results_1976.insert(len(df_results_1976.columns), 'RunnerUpVotes', np.zeros(612))
df_results_1976.insert(len(df_results_1976.columns), 'WinnerVoteShare', np.zeros(612).astype('float'))
df_results_1976 = df_results_1976.replace('D.C.','District of Columbia')
df_results_1976

,Year,State,Population,Votes,VotesT,Party,WinnerVotes,RunnerUpVotes,WinnerVoteShare
0,1976,Alabama,0.0,9.0,0.0,D,0.0,0.0,0.0
1,1980,Alabama,0.0,9.0,0.0,R,0.0,0.0,0.0
2,1984,Alabama,0.0,9.0,0.0,R,0.0,0.0,0.0
3,1988,Alabama,0.0,9.0,0.0,R,0.0,0.0,0.0
4,1992,Alabama,0.0,9.0,0.0,R,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
607,2004,Wyoming,0.0,3.0,0.0,R,0.0,0.0,0.0
608,2008,Wyoming,0.0,3.0,0.0,R,0.0,0.0,0.0
609,2012,Wyoming,0.0,3.0,0.0,R,0.0,0.0,0.0
610,2016,Wyoming,0.0,3.0,0.0,R,0.0,0.0,0.0


In [22]:
pres_results = pd.read_csv('1976-2020-president.csv')
candidates_dict = {}
candidates_dict[1976] = {'D':'Carter','R':'Ford'}
candidates_dict[1980] = {'D':'Carter','R':'Reagan'}
candidates_dict[1984] = {'D':'Mondale','R':'Reagan'}
candidates_dict[1988] = {'D':'Dukakis','R':'Bush'}
candidates_dict[1992] = {'D':'Clinton','R':'Bush'}
candidates_dict[1996] = {'D':'Clinton','R':'Dole'}
candidates_dict[2000] = {'D':'Gore','R':'Bush'}
candidates_dict[2004] = {'D':'Kerry','R':'Bush'}
candidates_dict[2008] = {'D':'Obama','R':'McCain'}
candidates_dict[2012] = {'D':'Obama','R':'Romney'}
candidates_dict[2016] = {'D':'Clinton','R':'Trump'}
candidates_dict[2020] = {'D':'Biden','R':'Trump'}

In [23]:
ec_votes_dict = {}
ec_votes_dict[1976] = 1970
ec_votes_dict[1980] = 1970
ec_votes_dict[1984] = 1980
ec_votes_dict[1988] = 1980
ec_votes_dict[1992] = 1990
ec_votes_dict[1996] = 1990
ec_votes_dict[2000] = 1990
ec_votes_dict[2004] = 2000
ec_votes_dict[2008] = 2000
ec_votes_dict[2012] = 2010
ec_votes_dict[2016] = 2010
ec_votes_dict[2020] = 2010

In [24]:
state_pop = pd.read_csv('county_population.csv', encoding='latin-1')
state_pop2 = pd.read_csv('nst-est2019-alldata.csv')
ec_votes = pd.read_csv('ElectoralCollegeData.csv')

In [25]:
dc_2010_pop = state_pop2[state_pop2['NAME'] == 'District of Columbia']['POPESTIMATE2010'].iloc[0]
dc_2019_pop = state_pop2[state_pop2['NAME'] == 'District of Columbia']['POPESTIMATE2019'].iloc[0]
dc_growth_rate = ((dc_2019_pop/dc_2010_pop)-1)/9
dc_2020_pop = round(dc_2019_pop * np.exp(dc_growth_rate))

In [26]:
for index,row in df_results_1976.iterrows():
    # population
    if row['Year'] < 2010:
        column_str = 'pop'+str(row['Year'])
        df_results_1976.at[index,'Population'] = state_pop[(state_pop['county_fips'] == 0) & 
                                                           (state_pop['county_name'] == row['State'])][column_str].iloc[0]
    
    elif row['Year'] < 2020:
        column_str = 'POPESTIMATE'+str(row['Year'])
        df_results_1976.at[index,'Population'] = state_pop2[state_pop2['NAME'] == row['State']][column_str].iloc[0]
        
    else:
        if row['State'] == 'District of Columbia':
            df_results_1976.at[index,'Population'] = dc_2020_pop
        else:
            df_results_1976.at[index,'Population'] = ec_votes[(ec_votes['Year'] == 2020) &
                                                         (ec_votes['State'] == row['State'])]['Population'].iloc[0]
        
    # theoretical electoral votes
    target_year = ec_votes_dict[row['Year']]
    if row['State'] == 'District of Columbia':
        df_results.at[index,'VotesT'] = 1
    else:
        df_results_1976.at[index,'VotesT'] = ec_votes[(ec_votes['Year'] == target_year) & 
                                                 (ec_votes['State'] == row['State'])]['VotesP'].iloc[0]
    
    #election results
    state_year_result = pres_results[(pres_results['year'] == row['Year']) & 
                                    (pres_results['state'] == row['State'].upper()) &
                                    (pres_results['candidate'] != False)]
    
    names = state_year_result['candidate'].str.split(', ', 1, True)
    state_year_result = pd.concat([state_year_result, names[0]], axis=1)
    year = row['Year']
    dem_votes = state_year_result[state_year_result[0] == candidates_dict[year]['D'].upper()]['candidatevotes'].sum()
    rep_votes = state_year_result[state_year_result[0] == candidates_dict[year]['R'].upper()]['candidatevotes'].sum()
    
    
    if row['Party'] == 'D':
        df_results_1976.at[index,'WinnerVotes'] = dem_votes
        df_results_1976.at[index,'RunnerUpVotes'] = rep_votes
        df_results_1976.at[index,'WinnerVoteShare'] = float(dem_votes / (dem_votes + rep_votes))
        
    else:
        df_results_1976.at[index,'WinnerVotes'] = rep_votes
        df_results_1976.at[index,'RunnerUpVotes'] = dem_votes
        df_results_1976.at[index,'WinnerVoteShare'] = float(rep_votes / (dem_votes + rep_votes))

In [29]:
columns = ['Population','Votes','VotesT','WinnerVotes','RunnerUpVotes']
for column in columns:
    df_results_1976[column] = df_results_1976[column].apply(np.int64)
df_results_1976

,Year,State,Population,Votes,VotesT,Party,WinnerVotes,RunnerUpVotes,WinnerVoteShare
0,1976,Alabama,3737000,9,9,D,659170,504070,0.566667
1,1980,Alabama,3894025,9,9,R,654192,636730,0.506763
2,1984,Alabama,3951824,9,9,R,872849,551899,0.612634
3,1988,Alabama,4023842,9,9,R,815576,549506,0.597456
4,1992,Alabama,4139269,9,9,R,804283,690080,0.538211
...,...,...,...,...,...,...,...,...,...
607,2004,Wyoming,502988,3,1,R,167629,70776,0.703127
608,2008,Wyoming,532981,3,1,R,164958,82868,0.665620
609,2012,Wyoming,576305,3,1,R,170962,69286,0.711606
610,2016,Wyoming,584215,3,1,R,174419,55973,0.757053
